In [53]:
import bs4 as bs
import pandas as pd
import matplotlib as plt
from matplotlib import pyplot as pplt
import os 
import pprint as p

#Die Buch-Kürzel des alten Testaments
AT = ["Gn", "Ex", "Lv", "Nm", "Dt", "Ios", "Idc", "Rt", "1Sm", "2Sm", "3Rg", "4Rg", "1Par", "2Par", "Esr", "Neh", "Tb",
      "Idt", "Est", "1Mcc"
    , "2Mcc", "3Mcc", "4Mcc", "Iob", "Ps", "Prv", "Ecl", "Ct", "Sap", "Sir", "Is", "Ier", "Lam", "Bar", "Ez", "Dn",
      "Os", "Ioel", "Am", "Abd"
    , "Ion", "Mi", "Na", "Hab", "So", "Agg", "Za", "Mal"]


  
def percent_calc(num1, num2):
    '''returns percentage value'''
    return (num1/num2)*100
    
 
    
def analyse(filename):
    '''Opens a file specified by the filename, including the directory and creates dataframe for the respective file. Subsequently, this function analyses the chapter with regard to our criteria'''
    file_title = filename.split(".")[0]
    with open(filename, 'r') as f:
        file = f.read()   
    soup = bs.BeautifulSoup(file, 'lxml')
    Refs = soup.select('ref[cRef]')

    data = []
    for ref in Refs:
        if ref.get('cref').split('_')[0] in AT:
            if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
                data.append((ref.get('cref') , 'at', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
            else:
                data.append((ref.get('cref'), 'at', ref.getText(), 'not a direct quote'))
        else:
            if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
                data.append((ref.get('cref') , 'nt', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
            else:
                data.append((ref.get('cref'), 'nt', ref.getText(), 'not a direct quote'))
    #print(data)
    # der eigentliche Dataframe wird gesetzt
    df_data = pd.DataFrame(data)
    # der Header wird gesetzt.
    df_data.columns = ['kuerzel', 'atnt', 'stelle', 'zitat']



    # Hier wird eine Liste der wörtlichen Zitate erstellt
    qs = []
    for ref in Refs:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling("q"):
            qs.append(ref.parent.find_previous_sibling("q").get_text(" ", strip=True))


    # Zählen der Gesamtlänge des zitierten Textes; one Leerzeilen und Zeilenumbrüche 
    quote_len = 0
    for i in range(len(qs)):
        quote_len +=len(qs[i].replace(" ","").replace("\n", ""))
        i+=1



    ges = bs.BeautifulSoup(file, 'lxml')

    ges_soup = ges.select('div')
    total_len=0
    for i in range(len(ges_soup)):
        total_len += len(ges_soup[i].getText().replace(" ", "").replace("n", ""))
    
    print(f"Anzahl an erfassten Bibelstellen: {len(Refs)}")
    print(f"wörtliche Zitate: {len(qs)}")
    print("Gesamtlänge:" + str(total_len))
    print("Zitatlänge :"+str(quote_len))    
    print("Prozentualer Anteil: " +(str(percent_calc(quote_len, total_len))))
    lines = [f"Anzahl an erfassten Bibelstellen: {len(Refs)}", f"wörtliche Zitate: {len(qs)}", f"Gesamtlänge: {str(total_len)}", f"Zitatlänge :{str(quote_len)}",f"Prozentualer Anteil: {(str(percent_calc(quote_len, total_len)))}"]
    with open(f'Visualisierungen/{file_title}', 'w') as f2:
        for line in lines:
            f2.write(line)
            f2.write("\n")
    print(df_data["kuerzel"].value_counts().plot(figsize=(8,8), kind="pie", legend=False, title=f"{filename}"))
    pplt.savefig(f'Visualisierungen/{file_title}.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["atnt"].value_counts().plot(figsize=(8,8), kind="bar", legend=False, title=f"{filename}"))
    pplt.savefig(f'Visualisierungen/{file_title}_atnt.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["kuerzel"].value_counts()[:5].plot(figsize=(8,8), kind="barh", legend=False, title=f"{filename}  @cRef)", color="purple"))
    pplt.savefig(f'Visualisierungen/{file_title}_cref.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["stelle"].value_counts()[:10].plot(figsize=(8,8), kind="barh", legend=False, title=f"{filename} nach Stellen", color="red"))
    pplt.savefig(f'Visualisierungen/{file_title}_nach_stellen.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    

# Daten für das gesamte Interim

In [50]:
#Probe-Sachen
with open(f'interim.xml', 'r') as f:
    file = f.read()
soup = bs.BeautifulSoup(file, 'lxml')
Refs = soup.select('ref[cRef]')

# Eine leere Liste wird erstellt und mit den Bibelstellen gefüllt. Hierbei wird sortiert, ob sie im Alten Testament stehen oder nicht.
# Ein zweites Auswahlkriterium ist die Frage, ob direkt vor dem Eltern-ELement des <ref type='biblical'> ein <q> steht, und somit ein direktes Zitat vorliegt.
data = []
for ref in Refs:
    if ref.get('cref').split('_')[0] in AT:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
            data.append((ref.get('cref') , 'at', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
        else:
            data.append((ref.get('cref'), 'at', ref.getText(), 'not a direct quote'))
    else:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
            data.append((ref.get('cref') , 'nt', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
        else:
            data.append((ref.get('cref'), 'nt', ref.getText(), 'not a direct quote'))
print(data)
# der eigentliche Dataframe wird gesetzt
df_data = pd.DataFrame(data)
# der Header wird gesetzt.
df_data.columns = ['kuerzel', 'atnt', 'stelle', 'zitat']



stellen = []
for i in range (0, len(df_data)):
    #if df_data['zitat'][i] != 'not a direct quote':
    stellen.append(df_data['stelle'][i])


# Hier wird eine Liste der wörtlichen Zitate erstellt
qs = []
for ref in Refs:
    if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling("q"):
        qs.append(ref.parent.find_previous_sibling("q").get_text(" ", strip=True))


# Zählen der Gesamtlänge des zitierten Textes; one Leerzeilen und Zeilenumbrüche
quote_len = 0
for i in range(len(qs)):
    quote_len +=len(qs[i].replace(" ","").replace("\n", ""))
    i+=1



ges = bs.BeautifulSoup(file, 'lxml')

ges_soup = ges.select('div')
total_len=0
for i in range(len(ges_soup)):
    total_len += len(ges_soup[i].getText().replace(" ", "").replace("n", ""))

lines = [f"Anzahl an erfassten Bibelstellen: {len(Refs)}", f"wörtliche Zitate: {len(qs)}", f"Gesamtlänge: {str(total_len)}", f"Zitatlänge :{str(quote_len)}",f"Prozentualer Anteil: {(str(percent_calc(quote_len, total_len)))}"]
with open('Visualisierungen/interim.txt', 'w') as f2:
    for line in lines:
        f2.write(line)
        f2.write("\n")

print(f"Anzahl an erfassten Bibelstellen: {len(Refs)}")
print(f"wörtliche Zitate: {len(qs)}")
print("Gesamtlänge:" + str(total_len))
print("Zitatlänge :"+str(quote_len))
print("Prozentualer Anteil: " +(str(percent_calc(quote_len, total_len))))



[('Gn_1,26-27', 'at', 'Gen 1,26f.', 'not a direct quote'), ('Sir_15,14', 'at', 'Sir 15,14', 'hat in Gott gelassen inn der hand seines eig - nen Raths'), ('Rm_5,12', 'nt', 'Röm 5,12', 'not a direct quote'), ('Rm_5,15-19', 'nt', '15-19', 'not a direct quote'), ('Eph_2,1-3', 'nt', 'Eph 2,1-3', 'not a direct quote'), ('Gal_5,17', 'nt', 'Gal 5,17', 'not a direct quote'), ('Rm_5,12', 'nt', 'Röm 5,12', 'not a direct quote'), ('Rm_5,15-19', 'nt', '15-19', 'not a direct quote'), ('Iob_25,4-6', 'at', 'Hiob 25,4-6', 'not a direct quote'), ('1Cor_2,14', 'nt', '1 Kor 2,14', 'not a direct quote'), ('1Cor_5,1-5', 'nt', '5,1-5', 'not a direct quote'), ('Rm_5,20-21', 'nt', 'Röm 5,20f.', 'not a direct quote'), ('Rm_8,7', 'nt', '8,7', 'not a direct quote'), ('Rm_5,12', 'nt', 'Röm 5,12', 'die sünde ist durch eynen Roma. 5. menschen inn die welt kommen unnd durch die sünde der tod'), ('Eph_2,1-3', 'nt', 'Eph 2,1-3', 'not a direct quote'), ('Is_66,24', 'at', 'Jes 66,24', 'not a direct quote'), ('Io_3,16-17'

Anzahl an erfassten Bibelstellen: 283
wörtliche Zitate: 114
Gesamtlänge:204612
Zitatlänge :9713
Prozentualer Anteil: 4.747033409575196


In [65]:
analyse('interim_26/ch_10.xml')

Anzahl an erfassten Bibelstellen: 10
wörtliche Zitate: 6
Gesamtlänge:2140
Zitatlänge :732
Prozentualer Anteil: 34.205607476635514
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)


In [70]:
for i in range(1,27):
    if i < 10:
        print(f"analyse('interim_26/ch_0{i}.xml')")
    else:
        print(f"analyse('interim_26/ch_{i}.xml')")

analyse('interim_26/ch_01.xml')
analyse('interim_26/ch_02.xml')
analyse('interim_26/ch_03.xml')
analyse('interim_26/ch_04.xml')
analyse('interim_26/ch_05.xml')
analyse('interim_26/ch_06.xml')
analyse('interim_26/ch_07.xml')
analyse('interim_26/ch_08.xml')
analyse('interim_26/ch_09.xml')
analyse('interim_26/ch_10.xml')
analyse('interim_26/ch_11.xml')
analyse('interim_26/ch_12.xml')
analyse('interim_26/ch_13.xml')
analyse('interim_26/ch_14.xml')
analyse('interim_26/ch_15.xml')
analyse('interim_26/ch_16.xml')
analyse('interim_26/ch_17.xml')
analyse('interim_26/ch_18.xml')
analyse('interim_26/ch_19.xml')
analyse('interim_26/ch_20.xml')
analyse('interim_26/ch_21.xml')
analyse('interim_26/ch_22.xml')
analyse('interim_26/ch_23.xml')
analyse('interim_26/ch_24.xml')
analyse('interim_26/ch_25.xml')
analyse('interim_26/ch_26.xml')


In [71]:
analyse('interim_26/ch_01.xml')
analyse('interim_26/ch_02.xml')
analyse('interim_26/ch_03.xml')
analyse('interim_26/ch_04.xml')
analyse('interim_26/ch_05.xml')
analyse('interim_26/ch_06.xml')
analyse('interim_26/ch_07.xml')
analyse('interim_26/ch_08.xml')
analyse('interim_26/ch_09.xml')
analyse('interim_26/ch_10.xml')
analyse('interim_26/ch_11.xml')
analyse('interim_26/ch_12.xml')
analyse('interim_26/ch_13.xml')
analyse('interim_26/ch_14.xml')
analyse('interim_26/ch_15.xml')
analyse('interim_26/ch_16.xml')
analyse('interim_26/ch_17.xml')
analyse('interim_26/ch_18.xml')
analyse('interim_26/ch_19.xml')
analyse('interim_26/ch_20.xml')
analyse('interim_26/ch_21.xml')
analyse('interim_26/ch_22.xml')
analyse('interim_26/ch_23.xml')
analyse('interim_26/ch_24.xml')
analyse('interim_26/ch_26.xml')

Anzahl an erfassten Bibelstellen: 2
wörtliche Zitate: 1
Gesamtlänge:1107
Zitatlänge :45
Prozentualer Anteil: 4.0650406504065035
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
Anzahl an erfassten Bibelstellen: 8
wörtliche Zitate: 1
Gesamtlänge:1867
Zitatlänge :173
Prozentualer Anteil: 9.266202463845742
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
Anzahl an erfassten Bibelstellen: 13
wörtliche Zitate: 7
Gesamtlänge:3742
Zitatlänge :944
Prozentualer Anteil: 25.227151256012824
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
Anzahl an erfassten Bibelstellen: 3
wörtliche Zitate: 0
Gesamtlänge:617
Zitatlänge :0
Prozentualer Anteil: 0.0
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0

ValueError: Length mismatch: Expected axis has 0 elements, new values have 4 elements